In [ ]:
!mkdir dataset
!pip install gdown
!gdown 1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq
!mv refcocog.tar.gz ./dataset/
!ls dataset

mkdir: cannot create directory ‘dataset’: File exists
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Access denied with the following error:

 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1xijq32XfEm6FPhUb7RsZYWHc2UuwVkiq 

mv: cannot stat 'refcocog.tar.gz': No such file or directory
refcocog


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
     

import tarfile
data_dir='/content/drive/MyDrive/refcocog.tar.gz'
# Extract data
tar = tarfile.open(data_dir)
tar.extractall('dataset/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!tar -xf dataset/refcocog.tar.gz -C dataset
!ls dataset

tar: dataset/refcocog.tar.gz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
refcocog


In [ ]:
!pip install torch
!pip install torchvision
!pip install SummaryWriter
!pip install tensorboard
!pip install scikit-image
!pip install matplotlib
!pip install Lambda
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import os
from posixpath import split
import json
import tarfile
import io
import pickle
import sys
from PIL import Image
from collections import OrderedDict
from pathlib import Path
from collections import defaultdict


import numpy as np
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
from torchvision.utils import draw_bounding_boxes
import torch
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

device = "cuda" if torch.cuda.is_available() else "cpu"
temperature = 1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement SummaryWriter (from versions: none)
ERROR: No matching distribution found for SummaryWriter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-djsjca5b
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-djsjca5b
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done


In [ ]:
from clip import clip

model, preprocess = clip.load("RN50")
model = model.float().to(device).eval()

In [ ]:
class RefCOCOgDataset(Dataset):
    def __init__(self, dataset, transform, target_transform, data_dir='dataset/refcocog'):
        super(RefCOCOgDataset, self).__init__()
        self.data_dir = data_dir
        self.transform = transform
        self.target_transform = target_transform
        self.dataset = dataset
        self.encoded_captions = tokenizer(
            list(captions), padding=True, truncation=True, max_length=CFG.max_length
        )
        

    def __getitem__(self, idx): 
        data_item =  
        { key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()}
        fname = os.path.join(self.data_dir+'/images/', self.dataset[idx]['image'])
        #fname = os.path.join(self.data_dir+'/images/', list(self.dataset.items())[idx][0])
        image = Image.open(fname).convert('RGB')
        #texts = list(self.dataset.items())[idx][1]
        if self.transform:
            data_item['image'] = self.transform(image).float()
        if self.target_transform:
            data_item['captions'] = [sent for desc in self.dataset[idx]['captions'] for sent in desc]
        return data_item


    def __len__(self):
        return len(self.dataset)



In [ ]:
def get_datasets(refs, train = True):
  ref_data = []
  for val in refs:
    if train:
      if val['split'] == 'train' or val['split'] == 'val':
        ref_data.append(val)
    else:
      if val['split'] == 'test':
        ref_data.append(val)
  
  '''for i,v in enumerate(datasets):
    training_data, test_data = v'''

  return ref_data


def preProcess_datasets(data_dir, train):
  dataset = {}
  data = {}
  refs = {}     
  f = open(f'{data_dir}/annotations/refs(umd).p', 'rb')
  data['refs'] = pickle.load(f)
  if train:
    refs = get_datasets(data['refs'], train=True)
  else:
    refs = get_datasets(data['refs'], train=False)
  instances_file = os.path.join(f'{data_dir}/annotations/instances.json')
  instances = json.load(open(instances_file, 'r'))
  data['images'] = instances['images']
  data['annotations'] = instances['annotations'] 
  #print(data['refs'][0])
  #print(data['images'][0])
  #print(data['annotations'][0])
  for key,val in enumerate(refs):
    for v in data['images']:
      if(val['image_id'] == v['id']): 
        fname = os.path.join(data_dir+'/images/', v['file_name'])
        if os.path.exists(fname):
          dataset[key] = {}
          dataset[key]['image'] = v['file_name']
          dataset[key]['captions'] = [t['sent'] for t in val['sentences']]
          dataset[key]['id'] = val['image_id']
      else: 
        continue

  return dataset


  def visualise_result(img, txt):
    resize_image = T.Resize(100)
    img = resize_image(img)
    convert_tensor = T.ToTensor()
    image = convert_tensor(img).to(device)
    image = image.clone().detach()
    image = image.type(torch.ByteTensor).to(device)
    boxs = torch.tensor(w['bbox'],dtype=torch.int).to(device) 
    boxes = boxs.reshape([1,4])   
    boxes = torchvision.ops.box_convert(boxes, "cxcywh", "xyxy").to(device)
    image = draw_bounding_boxes(image=image, boxes=boxes, width=2, colors=(0,0,255), fill=True).to(device)
    image = image.permute(1,2,0)

In [ ]:
def encode_data(batch):
    # preprocess the images to transform from filenames to images to tensors
    '''images = [preprocess(Image.open(os.path.join(data_dir+'/images/',image))) for image in images_fp]

    # preprocess the texts to transform from text to tensors
    images = torch.tensor(np.stack(images)).to(device)

    text_tokens = clip.tokenize(["This is " + sent for desc in texts for sent in desc]).to(device)

    # encode the inputs
    with torch.no_grad():
        images_z = model.encode_image(images).float()
        texts_z = model.encode_text(text_tokens).float()
        texts_z /= texts_z.norm(dim=-1, keepdim=True)'''
    with torch.no_grad():
        # iterate over the test set
      #for batch_idx, batch in enumerate(data_loader):
      image_input = batch['image']
      text_input = batch['labels']
    # load data into GPU
      image_input = image_input.to(device)
      text_input = text_input.to(device)
      text_features_all = []
            
      text_features = model.encode_text(text_input).float()
      text_features /= text_features.norm(dim=-1, keepdim=True)
      text_features_all.append(text_features)

      # evaluate the mean representation
      text_features = torch.stack(text_features_all).mean(dim=0)

      # renormalise
      text_features /= text_features.norm(dim=-1, keepdim=True)

    # forward pass
    # these two lines are different from the "traditional" ones
      image_features = model.encode_image(image_input).float()
      image_features /= image_features.norm(dim=-1, keepdim=True)

    return image_features, text_features

In [ ]:
def cosine_similarity(tensor1: torch.Tensor, tensor2: torch.Tensor):
  # normalise the image and the text
  tensor1 /= tensor1.norm(dim=-1, keepdim=True)
  tensor2 /= tensor2.norm(dim=-1, keepdim=True)

  # evaluate the cosine similarity between the sets of features
  similarity = (tensor1 @ tensor2.T)

  return similarity.to(device)

In [ ]:
def training_step(net, data_loader, optimizer, cost_function, device='cuda'):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    # set the network to training mode
    net = net.float()
    net.train()

  # iterate over the training set
    for batch_idx, batch in enumerate(data_loader):
        
      #load data into GPU
      batch['image'] = batch['image'].to(device)
      batch['captions'] = batch['captions'].to(device)
        # forward pass
      outputs, targets = net(batch)

      # loss computation
      loss = cost_function(outputs, targets)

      # backward pass
      loss.backward()

      # parameters update
      optimizer.step()

      # gradients reset
      optimizer.zero_grad()

      # fetch prediction and loss value
      samples += batch['image'].shape[0]
      cumulative_loss += loss.item()
      _, predicted = outputs.max(dim=1) # max() returns (maximum_value, index_of_maximum_value)

      # compute training accuracy
      cumulative_accuracy += predicted.eq(targets).sum().item()

    return cumulative_loss / samples, cumulative_accuracy / samples * 100

def test_step(net, data_loader, cost_function, device='cuda'):
    samples = 0.0
    cumulative_loss = 0.0
    cumulative_accuracy = 0.0

    net = net.float()
    # set the network to evaluation mode
    net.eval() 

    # disable gradient computation (we are only testing, we do not want our model to be modified in this step!)
    with torch.no_grad():
        # iterate over the test set
      
      for batch_idx, batch in enumerate(data_loader):
      
        #load data into GPU
        batch['image'] = batch['image'].to(device)
        batch['captions'] = batch['captions'].to(device)
        
        # forward pass
        outputs, targets = net(batch)

        # loss computation
        loss = cost_function(outputs, targets)
        # fetch prediction and loss value
        samples += batch['image'].shape[0]
        cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
        _, predicted = outputs.max(1)

        # compute accuracy
        cumulative_accuracy += predicted.eq(targets).sum().item()


    return cumulative_loss / samples, cumulative_accuracy / samples * 100

In [ ]:
def get_cost_function():
  cost_function = torch.nn.CrossEntropyLoss()
  return cost_function

In [ ]:
def get_optimizer(model, lr, wd, momentum):
  optimizer = torch.optim.SGD([
      {'params': model.classifier.parameters(), 'lr': lr}
  ], lr=lr / 10, weight_decay=wd, momentum=momentum)
  
  return optimizer

In [ ]:
class TextEncoder(nn.Module):
    def __init__(self, model_name=CFG.text_encoder_model, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()
        if pretrained:
            self.model = DistilBertModel.from_pretrained(model_name)
        else:
            self.model = DistilBertModel(config=DistilBertConfig())
            
        for p in self.model.parameters():
            p.requires_grad = trainable

        # we are using the CLS token hidden representation as the sentence's embedding
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = output.last_hidden_state
        return last_hidden_state[:, self.target_token_idx, :]

In [ ]:
"""
Applies Batch Normalization over a 1D input (or 2D tensor)

Shape:
  Input: (N, C)
  Output: (N, C)

Input Parameters:
  in_features: number of features of the input activations
  track_running_stats: whether to keep track of running mean and std. (default: True)
  affine: whether to scale and shift the normalized activations. (default: True)
  momentum: the momentum value for the moving average. (default: 0.9)

Usage:
  >>> # with learable parameters
  >>> bn = BatchNorm1d(4)
  >>> # without learable parameters
  >>> bn = BatchNorm1d(4, affine=False)
  >>> input = torch.rand(10, 4)
  >>> out = bn(input)
"""

class BatchNorm1d(torch.nn.Module):
  def __init__(self, in_features, track_running_stats=True, affine=True, momentum=0.9):
    super().__init__()
    
    self.in_features = in_features
    self.track_running_stats = track_running_stats
    self.affine = affine
    self.momentum = momentum
    
    if self.affine:
      self.gamma = torch.nn.Parameter(torch.ones(self.in_features, 1))
      self.beta = torch.nn.Parameter(torch.zeros(self.in_features, 1))
    
    if self.track_running_stats:
      # register_buffer registers a tensor as a buffer that will be saved as part of the model
      # but which does not require to be trained, differently from nn.Parameter
      self.register_buffer('running_mean', torch.zeros(self.in_features, 1))
      self.register_buffer('running_std', torch.ones(self.in_features, 1))
  
  def forward(self, x):
    # transpose (N, C) to (C, N)
    x = x.transpose(0, 1).contiguous().view(x.shape[1], -1)
    
    # calculate batch mean
    mean = x.mean(dim=1).view(-1, 1)
    
    # calculate batch std
    std = x.std(dim=1).view(-1, 1)
    
    # during training keep running statistics (moving average of mean and std)
    if self.training and self.track_running_stats:
      # no computational graph is necessary to be built for this computation
      with torch.no_grad():
        self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
        self.running_std = self.momentum * self.running_std + (1 - self.momentum) * std
    
    # during inference time
    if not self.training and self.track_running_stats:
      mean = self.running_mean
      std = self.running_std
    
    # normalize the input activations
    x = (x - mean) / std
    
    # scale and shift the normalized activations
    if self.affine:
      x = x * self.gamma + self.beta
    
    return x.transpose(0, 1)

In [ ]:
"""
Applies Batch Normalization over a 2D or 3D input (4D tensor)

Shape:
  Input: (N, C, H, W)
  Output: (N, C, H, W)

Input Parameters:
  in_features: number of features of the input activations
  track_running_stats: whether to keep track of running mean and std. (default: True)
  affine: whether to scale and shift the normalized activations. (default: True)
  momentum: the momentum value for the moving average. (default: 0.9)

Usage:
  >>> # with learable parameters
  >>> bn = BatchNorm2d(4)
  >>> # without learable parameters
  >>> bn = BatchNorm2d(4, affine=False)
  >>> input = torch.rand(10, 4, 5, 5)
  >>> out = bn(input)
"""

class BatchNorm2d(torch.nn.Module):
  def __init__(self, in_features, track_running_stats=True, affine=True, momentum=0.9):
    super().__init__()
    
    self.in_features = in_features
    self.track_running_stats = track_running_stats
    self.affine = affine
    self.momentum = momentum
    
    if self.affine:
      self.gamma = torch.nn.Parameter(torch.ones(self.in_features, 1))
      self.beta = torch.nn.Parameter(torch.zeros(self.in_features, 1))
    
    if self.track_running_stats:
      # register_buffer registers a tensor as a buffer that will be saved as part of the model
      # but which does not require to be trained, differently from nn.Parameter
      self.register_buffer('running_mean', torch.zeros(self.in_features, 1))
      self.register_buffer('running_std', torch.ones(self.in_features, 1))
  
  def forward(self, x):
    # transpose (N, C, H, W) to (C, N, H, W)
    x = x.transpose(0, 1)
    
    # store the shape
    c, bs, h, w = x.shape
    
    # collapse all dimensions except the 'channel' dimension
    x = x.contiguous().view(c, -1)
    
    # calculate batch mean
    mean = x.mean(dim=1).view(-1, 1)
    
    # calculate batch std
    std = x.std(dim=1).view(-1, 1)
    
    # keep running statistics (moving average of mean and std)
    if self.training and self.track_running_stats:
      with torch.no_grad():
        self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mean
        self.running_std = self.momentum * self.running_std + (1 - self.momentum) * std
    
    # during inference time
    if not self.training and self.track_running_stats:
      mean = self.running_mean
      std = self.running_std
    
    # normalize the input activations
    x = (x - mean) / std
    
    # scale and shift the normalized activations
    if self.affine:
      x = x * self.gamma + self.beta
    
    return x.view(c, bs, h, w).transpose(0, 1)


In [ ]:
def test_step_zero_shot_clip(net, data_loader, texts_z):
    samples = 0.0
    cumulative_accuracy = 0.0

    # set the network to evaluation mode
    net.eval()

    # disable gradient computation (we are only testing, we do not want our model to be modified in this step!)
    with torch.no_grad():
        # iterate over the test set
        for batch_idx, batch in enumerate(data_loader):
            image_input = batch['image']
            targets = batch['labels']
          # load data into GPU
            image_input = image_input.to(device)
            targets = targets.to(device)

            text_features_all = []
                  
            with torch.no_grad():
              text_features = model.encode_text(texts_z).float()
              text_features /= text_features.norm(dim=-1, keepdim=True)
              text_features_all.append(text_features)
    
            # evaluate the mean representation
            text_features = torch.stack(text_features_all).mean(dim=0)

            # renormalise
            text_features /= text_features.norm(dim=-1, keepdim=True)

          # forward pass
          # these two lines are different from the "traditional" ones
            image_features = model.encode_image(image_input).float()
            image_features /= image_features.norm(dim=-1, keepdim=True)
   
            outputs = (100 * image_features @ text_features.T).softmax(dim=-1)

            # fetch prediction and loss value
            samples += image_input.shape[0]
            _, predicted = outputs.max(1)

          # compute accuracy
            cumulative_accuracy += predicted.eq(targets).sum().item()
            print(cosine_similarity(images_z, texts_z))
            
    return cumulative_accuracy / samples * 100

In [ ]:
class CustomCLIP(torch.nn.Module):
  def __init__(self, num_classes: int = 10, bias=False):
    super().__init__()
    #model, _ = clip.load("RN50")
    in_features = 1024
    out_features = 1024
    # take the visual encoder of CLIP
    # we also convert it to be 32 bit (by default CLIP is 16)
    self.encoder = model.visual.float()
    #self.text_encoder = self.encode_text
    self.text_encoder = TextEncoder()

    #self.classifier = torch.nn.Linear(1024, num_classes)   
    # add a bottleneck
    self.classifier = torch.nn.Sequential(
      torch.nn.Linear(in_features, in_features // 2, bias=bias),
      torch.nn.BatchNorm1d(512),
      torch.nn.ReLU(inplace=True),
      torch.nn.Linear(in_features // 2, in_features // 2, bias=bias),
      torch.nn.BatchNorm1d(512),
      torch.nn.ReLU(inplace=True),
      torch.nn.Linear(in_features // 2, out_features, bias=bias),
      torch.nn.BatchNorm1d(1024),
      torch.nn.Sigmoid()
    )

  def encode_text(self, text):
      x = model.token_embedding(text).float()
      x = x + model.positional_embedding.float()
      x = x.permute(1, 0, 2)  # NLD -> LND
      #x = x.type(torch.float16).to(device)
      x = model.transformer(x).float()
      x = x.permute(1, 0, 2)  # LND -> NLD
      x = model.ln_final(x).float()

      # x.shape = [batch_size, n_ctx, transformer.width]
      # take features from the eot embedding (eot_token is the highest number in each sequence)
      x = x[torch.arange(x.shape[0]), text.argmax(dim=-1)] @ model.text_projection

      return x


  def forward(self, batch): 
    x = batch['image']
    x = self.encoder(x)
    x = self.classifier(x)
    y = batch['captions']
    y = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
    y = self.text_encoder(y)
    y = self.classifier(y)

    target_len = y.size(0) - x.size(0)
    x = F.pad(x, (0,0,0,target_len), value=0)
    x_similarity = cosine_similarity(x,x)
    y_similarity = cosine_similarity(y,y)


    targets = F.softmax(
      (x_similarity + y_similarity) / 2 * temperature, dim=-1
    )

    target= torch.argmax(targets ,axis=1)
    outputs = cosine_similarity(x,y)


    return outputs, target

In [ ]:
def pad_sequence(batch):
  return torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0)
def my_collate_fn(batch):
  '''captions = captions.squeeze(0).detach().cpu().numpy()
  captions = captions[:,:,0]
  print(captions.shape)
  captions = captions.permute(2,1,0)
  print(captions.shape)
  captions = captions.float().mean(2)'''
  return {
      'image': torch.stack([x['image'] for x in batch]),
      'captions': torch.cat([x['captions'] for x in batch], dim=0)
      }


def get_data(data_dir, batch_size=32, transform=True, target_transform=True, test_batch_size=32):

  
    if transform:
        # convert the PIL images to Tensors
        transform = preprocess
    else:
          # prepare data transformations and then combine them sequentially
        transform = T.Compose([torchvision.transforms.ToTensor()]) 
    if target_transform:   
        target_transform = T.Compose([
                                 lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)])
    else:
        target_transform = None

  # load data
    dataset = preProcess_datasets(data_dir, train=True)
    full_training_data = RefCOCOgDataset(dataset=dataset, transform=transform, target_transform=target_transform, data_dir=data_dir)

    test_dataset = preProcess_datasets(data_dir, train=False)
    test_data = RefCOCOgDataset(dataset=test_dataset, transform=transform, target_transform=target_transform, data_dir=data_dir)
  


    evens = list(range(0, len(full_training_data), 2))
    training_data2 = torch.utils.data.Subset(full_training_data, evens)


  # create train and validation splits
    num_samples = len(training_data2)
    training_samples = int(num_samples * 0.5 + 1)
    validation_samples = num_samples - training_samples

    training_data, validation_data = torch.utils.data.random_split(training_data2, [training_samples, validation_samples])

  # initialize dataloaders_collate
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=2, collate_fn=my_collate_fn)
    val_loader = torch.utils.data.DataLoader(validation_data, test_batch_size, shuffle=False, num_workers=2, collate_fn=my_collate_fn)
    test_loader = torch.utils.data.DataLoader(test_data, test_batch_size, shuffle=False, num_workers=2, collate_fn=my_collate_fn)

    print(len(training_data))

    print(len(validation_data))
    print(len(test_data))

    # pre-train model on zero-shot transfer learning
    #test_accuracy = test_step_zero_shot_clip(model, test_loader, texts_z)


    # evaluate accuracy on zero-shot learning

    #print("Test accuracy {:.2f}".format(test_accuracy))

    return train_loader, val_loader, test_loader
  

In [ ]:
# tensorboard logging utilities
def log_values(writer, step, loss, accuracy, prefix):
  writer.add_scalar(f"{prefix}/loss", loss, step)
  writer.add_scalar(f"{prefix}/accuracy", accuracy, step)

In [ ]:
# main funcition
def main(
      root='/content/dataset/refcocog/',
      data_dir='dataset/refcocog',
      batch_size=32,
      num_classes=10,
      learning_rate=0.01,
      weight_decay=0.000001,
      momentum=0.9,
      epochs=10,
    ): 
  writer = SummaryWriter(log_dir="runs/exp1")

  # train clip on zero-shot learning and instantiates dataloaders
  train_loader, val_loader, test_loader = get_data(data_dir=data_dir, batch_size=batch_size, transform=True, target_transform=True, test_batch_size=32)
  #dataset = get_data(data_dir=data_dir, batch_size=batch_size, transform=True, test_batch_size=None)

  # instantiate the network and move it to the chosen device (GPU)
  modified_model = CustomCLIP(num_classes=num_classes).to(device)
  
  # instantiate the optimizer
  optimizer = get_optimizer(modified_model, learning_rate, weight_decay, momentum)
  
  # define the cost function
  cost_function = get_cost_function()

  # evaluate accuracy of modified model on zero-shot learning
  #test_accuracy = test_step_zero_shot_clip(model, test_loader, texts_z).to(device)
  
  #print(cosine_similarity(images_z, texts_z))
  #print("Test accuracy {:.2f}".format(test_accuracy))

  # computes evaluation results before training
  print('Before training:')
  train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
  val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
  test_loss, test_accuracy = test_step(modified_model, test_loader, cost_function)

  # log to TensorBoard
  log_values(writer, -1, train_loss, train_accuracy, "train")
  log_values(writer, -1, val_loss, val_accuracy, "validation")
  log_values(writer, -1, test_loss, test_accuracy, "test")

  print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  # for each epoch, train the network and then compute evaluation results
  for e in range(epochs):
    
    train_loss, train_accuracy = training_step(modified_model, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)

    # logs to TensorBoard
    log_values(writer, e, val_loss, val_accuracy, "Validation")

    print('Epoch: {:d}'.format(e+1))
    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')

    # compute final evaluation results
    print('After training:')
    train_loss, train_accuracy = test_step(modified_model, train_loader, cost_function)
    val_loss, val_accuracy = test_step(modified_model, val_loader, cost_function)
    test_loss, test_accuracy = test_step(modified_model, test_loader, cost_function)

    # log to TensorBoard
    log_values(writer, epochs, train_loss, train_accuracy, "train")
    log_values(writer, epochs, val_loss, val_accuracy, "validation")
    log_values(writer, epochs, test_loss, test_accuracy, "test")

    print('\tTraining loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\tValidation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('\tTest loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
    print('-----------------------------------------------------')

  # closes the logger
  writer.close()


In [ ]:
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:<1024>"
os.environ["CUDA_LAUNCH_BLOCKING"]="1"

In [ ]:
main()

11201
11199
5023
Before training:


In [ ]:
import torch.nn.functional as F
data = torch.ones(4, 5)
data2 = torch.zeros(5,6)
print(data.shape)
#len = data2.size(0)-data.size(0)
# pad(left, right, top, bottom)
data = F.pad(data, (0,0,0,1),  value=0)
print(data.shape)

torch.Size([4, 5])
torch.Size([5, 5])
